In [82]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from rich import print
import torch
from model import BasicMLP

In [83]:
class LocalWindFieldDataset(Dataset):
    def __init__(self, csv_file, rect_fol, wind_fol, root_dir="", local=2):
        print("Local field set to: ", local)
        self.main_data = pd.read_csv(os.path.join(root_dir, csv_file))
        self.rect_file_path = os.path.join(root_dir, rect_fol)
        self.wind_file_path = os.path.join(root_dir, wind_fol)
        self.local = local
    def __len__(self):
        return len(self.main_data)
    
    def getLocalWinds(self,robo_coords, winds):
        startcoord = robo_coords - self.local
        endcoord = robo_coords + self.local + 1
        if startcoord[0] < 0 or startcoord[1] < 0 or endcoord[0] >= winds.shape[0] or endcoord[1] >= winds.shape[1]:
            print("Out of bounds")
            return None
        local_winds = winds[startcoord[0]:endcoord[0], startcoord[1]:endcoord[1]]
        return torch.tensor(local_winds,dtype=torch.float64) 
    
    def __getitem__(self, idx):
        data = self.main_data.iloc[idx]
        rect_path = os.path.join(self.rect_file_path, str(data["index"]) + "_r.npy")
        winds_path = os.path.join(self.wind_file_path, str(data["index"]) + "_m.npy")
        winds = np.load(winds_path)
        robo_coords = torch.tensor([data["xr"], data["yr"]])
        print(f"Robot Coordinates: {robo_coords}")
        local_winds = self.getLocalWinds(robo_coords, winds)
        rect = np.load(rect_path)
        # lidar = getLidar(rects)
        lidar = torch.rand(360,dtype=torch.float64)
        return lidar,torch.tensor(winds[robo_coords[0]][robo_coords[0]],dtype=torch.float64),local_winds

In [84]:
local_wind_field_dataset = LocalWindFieldDataset(csv_file="data.csv",rect_fol = 'rects',wind_fol = 'wind_fields', root_dir = 'data',local=1)

Local field set to:  1

In [85]:
model = BasicMLP(local = local_wind_field_dataset.local)

In [86]:
for i, sample in enumerate(local_wind_field_dataset, 1):
    lidar, wind_at_robot, winds_y = sample
    print(f"{i}:  lidar shape - {lidar.shape} and wind on robot -  {wind_at_robot.shape} and  \n Wind Fields shape- {winds_y.shape}")

Robot Coordinates: tensor([10, 10])

1:  lidar shape - torch.Size([360]) and wind on robot -  torch.Size([2]) and  
 Wind Fields shape- torch.Size([3, 3, 2])

Robot Coordinates: tensor([20, 20])

2:  lidar shape - torch.Size([360]) and wind on robot -  torch.Size([2]) and  
 Wind Fields shape- torch.Size([3, 3, 2])

Robot Coordinates: tensor([10, 30])

3:  lidar shape - torch.Size([360]) and wind on robot -  torch.Size([2]) and  
 Wind Fields shape- torch.Size([3, 3, 2])

Robot Coordinates: tensor([15, 21])

4:  lidar shape - torch.Size([360]) and wind on robot -  torch.Size([2]) and  
 Wind Fields shape- torch.Size([3, 3, 2])

In [87]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [88]:
for epoch in range(10):  # number of epochs
    model.train()
    running_loss = 0.0
    for i, sample in enumerate(local_wind_field_dataset, 1):
        lidar, wind_at_robot, winds_y = sample
        optimizer.zero_grad()
        
        # Ensure all tensors have the same dtype
        lidar = lidar.float()
        wind_at_robot = wind_at_robot.float()
        winds_y = winds_y.float()
        
        outputs = model(torch.cat((lidar, wind_at_robot.flatten()), dim=0))
        print(f"Output shape: {outputs.shape}, y {winds_y.shape}")
        loss = criterion(outputs, winds_y)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        print(f"Sample [{i}/{len(local_wind_field_dataset)}], Loss: {loss.item():.4f}")
    
    print(f"Epoch [{epoch+1}/10], Average Loss: {running_loss/len(local_wind_field_dataset):.4f}")

Robot Coordinates: tensor([10, 10])

Output shape: torch.Size([1, 3, 3, 2]), y torch.Size([3, 3, 2])

c:\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([3, 3, 2])) that is different to the input size (torch.Size([1, 3, 3, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Sample [1/4], Loss: 3639.0308

Robot Coordinates: tensor([20, 20])

...

Sample [3/4], Loss: 645.2345

Robot Coordinates: tensor([15, 21])

Output shape: torch.Size([1, 3, 3, 2]), y torch.Size([3, 3, 2])

Sample [4/4], Loss: 766.9377

Epoch [10/10], Average Loss: 800.8588